In [ ]:
!pip install "pymongo[srv]"

In [ ]:
import pymongo
from pymongo import MongoClient

In [ ]:
Image_connection = MongoClient("mongodb+srv://Govindu:<pwd>@week9.8ckx0.mongodb.net/?retryWrites=true&w=majority&appName=Week9")

In [ ]:
db = Image_connection["CBIR"]

In [ ]:
try:
  print(db.list_collection_names())
except Exception as e:
    print(f"Connection Failed: {e}")

['processed_images', 'image_metadata']


In [ ]:
# Let's load the image metadata into the image metadata collection
# Define the collection
image_metadata_collection = db["image_metadata"]  # Your collection name

import json

# Load JSON metadata file
with open('/content/image_metadata.json') as file:
    image_data = json.load(file)

# Insert data into MongoDB collection
if isinstance(image_data, list):
    image_metadata_collection.insert_many(image_data)  # For a list of documents
else:
    image_metadata_collection.insert_one(image_data)  # For a single document


In [ ]:
# Let's do loading of the processed images to the processed image collection
# Define the collection
processed_collection = db["processed_images"]  # Your collection name

import os
from bson import Binary

# Load and insert images
image_folder_path = '/content/Images'

for filename in os.listdir(image_folder_path):
    if filename.endswith(".png") or filename.endswith(".jpg"):
        with open(os.path.join(image_folder_path, filename), "rb") as image_file:
            binary_image = Binary(image_file.read())
            image_doc = {
                "filename": filename,
                "image_data": binary_image
            }
            processed_collection.insert_one(image_doc)


In [ ]:
# Example query to find an image by filename
query = {'filename': "Image12.jpg"}
image_document = processed_collection.find_one(query)

if image_document:
    print("Image found:", image_document["filename"])
else:
    print("No image found with the specified criteria.")


Image found: Image12.jpg


In [ ]:
Text_connection = MongoClient("mongodb+srv://Govindu:<pwd>@week9.8ckx0.mongodb.net/?retryWrites=true&w=majority&appName=Week9")

In [ ]:
db = Text_connection["NLP"]  # database name

# Test the connection by listing collections (optional)
try:
    print(db.list_collection_names())
except Exception as e:
    print(f"Connection failed: {e}")

[]


In [ ]:
# Define the collection
doc_metadata_collection = db["doc_metadata"]  # collection name

import json

# Load JSON metadata file
with open("/content/text_metadata.json") as file:
    doc_data = json.load(file)

# Insert data into MongoDB collection
if isinstance(doc_data, list):
    doc_metadata_collection.insert_many(doc_data)  # For a list of documents
else:
    doc_metadata_collection.insert_one(doc_data)  # For a single document

In [ ]:
# Define the collection
processed_doc_collection = db["Processed_doc"]  # Your collection name

import os
from datetime import datetime

# Specify folder containing your documents
document_folder_path = "/content/Documents"

# Iterate over all files in the specified folder
for filename in os.listdir(document_folder_path):
    file_path = os.path.join(document_folder_path, filename)

    # Ensure we only process files (skip directories)
    if os.path.isfile(file_path):
        with open(file_path, "r") as file:
            # Read the contents of the file as text
            file_content = file.read()

            # Create a document for insertion into MongoDB
            document_data = {
                "filename": filename,
                "content": file_content,
                "metadata": {
                    "processed_date": datetime.utcnow().isoformat()
                },
                "tags": ["text", "document"]  # Add any relevant tags as needed
            }

            # Insert the document into the collection
            processed_collection.insert_one(document_data)

In [ ]:
from PIL import Image
import numpy as np

from skimage.feature import graycomatrix, graycoprops
from skimage.color import rgb2gray

im_frame = Image.open('/content/fruits.jpg')
image = (255*rgb2gray(np.array(im_frame))).astype(np.uint8)  # 'Your_2D_np_array'

# Generate GLCM
distances = [50]  # Offset
angles = [np.pi/2]  # Vertical Direction
glcm = graycomatrix(image, distances=distances, angles=angles, levels=255)

# Calculate Features from GLCM
contrast = graycoprops(glcm, 'contrast')
dissimilarity = graycoprops(glcm, 'dissimilarity')
homogeneity = graycoprops(glcm, 'homogeneity')
energy = graycoprops(glcm, 'energy')
correlation = graycoprops(glcm, 'correlation')

In [ ]:
contrast

array([[4161.5949375]])